In [1]:
# import libraries

from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import io


Using TensorFlow backend.
/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# import data

#path = get_file('shakespeare-sonnets.txt', origin='https://www.dropbox.com/s/3wuwuc2ef36tdi0/shakespeare_sonnets.txt')
!wget https://www.dropbox.com/s/3wuwuc2ef36tdi0/shakespeare_sonnets.txt

with io.open('shakespeare_sonnets.txt', encoding='utf-8') as f:
    text = f.read().lower()
print('corpus length:', len(text))


--2018-09-10 23:04:25--  https://www.dropbox.com/s/3wuwuc2ef36tdi0/shakespeare_sonnets.txt
Resolving www.dropbox.com (www.dropbox.com)... 162.125.81.1, 2620:100:6031:1::a27d:5101
Connecting to www.dropbox.com (www.dropbox.com)|162.125.81.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/3wuwuc2ef36tdi0/shakespeare_sonnets.txt [following]
--2018-09-10 23:04:25--  https://www.dropbox.com/s/raw/3wuwuc2ef36tdi0/shakespeare_sonnets.txt
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucb88ce9a10c893d8cb88707dca3.dl.dropboxusercontent.com/cd/0/inline/AQLs4t6JcCRhNNipIwfZI9kfaKuWz6q9uB1mvVEaTeSm2AyhK80GzYTF5SFEYx9o9hwnAhW7oI1IUNmMFB4x5WwIiZtZ4P3D8S0vimWw41iUNqHYrJCQNV5bAhWp3AgSgtpvJYsuN409l-MLZMR5kos6i3mI9nbdhWkyr1z4Xm9O5e1G-jM1XFO64m5vpyKJzheETDszSI6gd26XKSvYKoyr/file [following]
--2018-09-10 23:04:26--  https://ucb88ce9a10c893d8cb88707dca3.dl.dropboxusercontent.com/cd/0/i

In [3]:
len(text)

102939

In [4]:
chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

('total chars:', 48)


In [5]:
# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

('nb sequences:', 34300)


In [6]:
print(sentences[0])
print(sentences[1])
print(sentences[2])
print(sentences[3])

the sonnets

by william shakespeare

   
 sonnets

by william shakespeare

      
nnets

by william shakespeare

         
ts

by william shakespeare

            


In [7]:
print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Vectorization...


In [8]:
# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

Build model...


In [9]:
optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

### Sampling some text from the model

In [10]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [11]:
def on_epoch_end(epoch, logs):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

In [12]:
#callback

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

In [13]:
model.fit(x, y,
          batch_size=128,
          epochs=60,
          callbacks=[print_callback])

Epoch 1/60
34176/34300 [============================>.] - ETA: 0s - loss: 2.4405()
----- Generating text after Epoch: 0
('----- diversity:', 0.2)
----- Generating with seed: "s of birds, nor the sweet smell
  of dif"
s of birds, nor the sweet smell
  of dife so beed the will the mend thee sered,
  i thee seat the beat se thou shen the will seat,
  thee sore the thee seat so thee the heall sare,
                          10
  whou the me se the will the mere se seat,
                          10
  in the the thee be wine so chee he sell,
  the thee the thee the will thou the mere,
  the hath the beare se the mere se the thee,
  whou here be the thee ()
('----- diversity:', 0.5)
----- Generating with seed: "s of birds, nor the sweet smell
  of dif"
s of birds, nor the sweet smell
  of difit sure i dound thougede or mengere,
  thou prise coll nough and the buth leou sast, and sowe,
  thou ure my thee wand sty the my thered,
  the hever wall halld though the me beant
  not bered the greas 

  thou i mane on grown thee stare, which change,
    th()
('----- diversity:', 1.0)
----- Generating with seed: "
  nor services to do till you require. "

  nor services to do till you require. wash dame,
  thou not mace the will asgent beart.
    a deaments manefless hig slows fainnt,
  and i thios, and i seaving i cagen
  and, t with not of you, dich the old'st wrens,
  you not bltathan 'doth decelvent a plond's preasu,
  unral sonder sounwers with thee is will.
  o thn mine me, as corvir hath poless will?
    jo ed every you love ied'st the  gain.


                      28
  fo mrols()
('----- diversity:', 1.2)
----- Generating with seed: "
  nor services to do till you require. "

  nor services to do till you require. and
  ot look, trrey peetteen sigharns,
    ellen in heling, illound arcuens not ntee,
  till uploonies have, ufhaven doth outh mether love,
  with, 'ice he scilemsh epceoue-sumely hearth vosing:
  'ick non tomed wo farstiring seillving hall-gri1ing this a grige.
 

34300/34300 [==============================] - 31s - loss: 1.3834    
Epoch 8/60
34176/34300 [============================>.] - ETA: 0s - loss: 1.3261()
----- Generating text after Epoch: 7
('----- diversity:', 0.2)
----- Generating with seed: "lse women's fashion,
  an eye more brigh"
lse women's fashion,
  an eye more bright to the praise the will,
                       143
  whon i do that shall the warth of thee,
    that should beauty the with my self be deced
  that i though the warth thy store to thee,
    that i do though my self doth the courd,
  and i dest in the beauty the world and ore,
    that should when i beauty stound me thee,
  and shall be the store with thy store be,
    thou be mine eye all the w()
('----- diversity:', 0.5)
----- Generating with seed: "lse women's fashion,
  an eye more brigh"
lse women's fashion,
  an eye more bright be do the world and coldst to content,
  the of my self so grace stould when i dide?
    thou by the with my seems i speck after lo

  of thy beauty stould neart have story,
  which stould that lovely of this store,
  that you should than blow thy heart so rearess,
  to leave of ()
('----- diversity:', 1.0)
----- Generating with seed: "isdain:
  lest sorrow lend me words and "
isdain:
  lest sorrow lend me words and find'race dist,
    both not in thus yeace me werts to bear be.
  ceel no dearly thy baths my selver.


                     48u
  ne carnt nuth thy reart for therefatrem,
  ray wey ther for would wad blass's stant,
  or ill the hat ne's recteck with siew truty,  
  ull warlsur nave flome hath buridient
  to reaplere wasteth by arest a or med:
  nor be they farth where forbortiled stound,
  i vars()
('----- diversity:', 1.2)
----- Generating with seed: "isdain:
  lest sorrow lend me words and "
isdain:
  lest sorrow lend me words and suemervenct kind,
  dosigladiel's fantary of at mory,
  y thou that my worth rectace a they, kispe sour
  so dape auth teave, love birsmilite '
  burileshe no reme; aceen di

/home/manishv/.local/lib/python2.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


...

  and summer eyes by true deed being ()
('----- diversity:', 0.5)
----- Generating with seed: "hen vouchsafe me but this loving thought"
hen vouchsafe me but this loving thought,
  and thy sun time beauty i, which stort:
  and the wiltic his compounds, ner was true,
  which still make he off being musted by true,
  which stance find ruikerness wheriof such shall hand cast in or be.
    for all thy life hath my pest from my rease,
    the world with some summer beauty ad more:
  then add of this self to self these bdour,
  that shall in hade the wiltike so, weed,
  thy re()
('----- diversity:', 1.0)
----- Generating with seed: "hen vouchsafe me but this loving thought"
hen vouchsafe me but this loving thought,
  many you so beauty words the best kind,
  whose thou thou stall shall to sundangy greatle.
    yor 'gassless rientarides eyes by tisse
  and chust sooury motion be brove i will.


        we hoursure-ors, morrign dresss fresold:
    the hacy with this fair it or beausy ime
  wit

  so are you to my thoughts as food thy heart,
  that you losse blake so ghack a coanted
  and spurene, cherkings asuromes of sight of thy willow,
  may shall by thou be, and therefore mine,
  thou by beauth thou dost birtts huld withink,
  which region apprimes that a keming hone ,
  i' muse again ho one sorluned but the take,
  which wilt shel in hided weewil only truth to more?
  not ill as well be give dearad, men of okige:
  for()
34300/34300 [==============================] - 30s - loss: 0.8138    
Epoch 48/60
34176/34300 [============================>.] - ETA: 0s - loss: 0.8141()
----- Generating text after Epoch: 47
('----- diversity:', 0.2)
----- Generating with seed: "o, neither he, nor his compeers by night"
o, neither he, nor his compeers by night.


                      48
  why with thy sundance thee behind at made.
    but far tongless first i are with my arth,
    then by thy sin, ampears thy sull by all.


                    148
  why hath praise that is this beauty 

  then in the warld with praises i to prose,
  the warthans his pront m()
('----- diversity:', 0.5)
----- Generating with seed: "mmon up remembrance of things past,
  i "
mmon up remembrance of things past,
  i see have the bes, nor mine, by thy percrows
  should making boundang the surle than precege.
    i diath where toward of the thee and prize.
    for it beauty hameging on store more hand.
  that shall and mensen of and makent spend
  then should and make and the warld with mook,
    and sumpre or chust to compoest in the warth spose,
  that she give thee mine eye storgs of bood.


                ()
('----- diversity:', 1.0)
----- Generating with seed: "mmon up remembrance of things past,
  i "
mmon up remembrance of things past,
  i guect in have gid inving the bleather see,
  than blom, or beauty is made me thanse.
    his robthough the dear keep of me wnowed
  which wonsuresong in prowes is hame,
  thy pil lovely newel bescanger when in measule;
  when in canst entol'st shoul

  or whose mine eppace as()
('----- diversity:', 1.2)
----- Generating with seed: "       14
  not from the stars do i my j"
       14
  not from the stars do i my joy,
  ush th' lile a lear thus pared, when that love,
  when all the batterning must compoun,
  and in the are of beauty other grawed,
  sapen as beauty in though sweet with made.
    uitise not favong that loving witnome.
  yet therefore is 'gain, to by eart broas,
  still soorly in thy slandy act old,
  thy rearts, and make thou knowiet to the too,
  to our munanes being which that hand be.
    ()
34300/34300 [==============================] - 30s - loss: 0.7762    
Epoch 55/60
34176/34300 [============================>.] - ETA: 0s - loss: 0.7800()
----- Generating text after Epoch: 54
('----- diversity:', 0.2)
----- Generating with seed: "pupil pen
  neither in inward worth nor "
pupil pen
  neither in inward worth nor belownt of good,
    but the jubger was bloovern of thy heart,
  thy suck age thy sulmy hust his good c

                      14l goth more woows of your say, and more:
  and all these for they stear did love's deepay,
  o whore will keep doth but surer was winter be.


         ()
('----- diversity:', 0.5)
----- Generating with seed: "  strikes each in each by mutual orderin"
  strikes each in each by mutual ordering:
  so shall ha'd men' seeling still the sellow,
  in my self away specked dient madow still steel,
  which is not false i have sweet his post,
  in heart that i wee so the trupt to thee.


                     128
  wro 'hiding fort, wibhest fortward bother wee,
  and all thou known mounst i say my true
  is this, and breathe chust thee where with his good,
  i must fair all.
    for all the war()
('----- diversity:', 1.0)
----- Generating with seed: "  strikes each in each by mutual orderin"
  strikes each in each by mutual ordering?
  o though to eve you were with belove,
  the four say thy perfect in love, thy face.
  giirs the watcance of thy proud jubgen tend,
  deat f